In [6]:
import re # new module required for dataframe creation
import numpy as np
import pandas as pd
import os
import math # required for prediction conversion

import tensorflow as tf
import tensorflow.keras
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

af_dir = '../../all_faces_bucket/'
disk_data_dir = '../../all_faces_disk/home/jupyter/forensics_split/'

In [ ]:
# Copy benchmark to disk
!mkdir /home/jupyter/ff_bench 
!gsutil -m cp -r gs://all_faces/benchmark_cropped_images /home/jupyter/ff_bench 

In [24]:
def get_model(architecture):
    '''Loads one of the saved models based on specified architecture'''
    
    return load_model(af_dir + 'trained_models/saved_models/' + architecture + '_model.h5')

def get_predictions(model):
    '''Takes in a loaded model and outputs filenames and multi-dimensional
    predictions for each class.
    
    Works by initiating an instance of ImageDataGenerator which is used for
    flow_from_directory method.'''
    # Normalise and centre test data
    datagen = ImageDataGenerator(samplewise_std_normalization=True, samplewise_center=True)
    generator = datagen.flow_from_directory('../../ff_bench', target_size=(224, 224),
                                            shuffle = False, batch_size=1, class_mode=None)
    filenames = generator.filenames
    nb_samples = len(filenames)
    generator.reset() # figure out this 
    predictions = model.predict(generator, steps = nb_samples, verbose=1, workers=8)
    
    return filenames, predictions

In [ ]:
with tf.device('/cpu:0'): # use if GPU is training and has no leftover memory
    loaded_model = get_model('mobilenet_new')
filenames, multidim_predictions = get_predictions(loaded_model)